In [ ]:
import polars as pl
import datetime as dt

In [ ]:
temp = (
    pl.read_excel('./municipios_pib.xlsx')
    .drop(['municipio_nome_uf'])
    .unpivot(index='municipio_codigo', variable_name='aaaa', value_name='pib_brl')
    .with_columns(
        pl.col('pib_brl').mul(1e3),
        pl.col('aaaa').str.to_date(format='%Y'),
        pl.col('municipio_codigo').str.slice(length=2, offset=0).alias('uf_codigo')
    )
    .rename({
        'municipio_codigo':'municipio_codigo_7'
    })
)

In [ ]:
(
    pl.read_excel('./municipios_pib.xlsx')
    .drop(['municipio_nome_uf'])
    .unpivot(index='municipio_codigo', variable_name='aaaa', value_name='pib_brl')
    .with_columns(
        pl.col('pib_brl').mul(1e3),
        pl.col('aaaa').str.to_date(format='%Y'),
        pl.col('municipio_codigo').str.slice(length=2, offset=0).alias('uf_codigo')
    )
    .rename({
        'municipio_codigo':'municipio_codigo_7'
    })
    .filter(
        pl.col('aaaa') == dt.datetime(2021,1,1)
    )
    .join(other=pl.read_excel('./uf_pib_2022.xlsx'), on='uf_codigo', how='left')
    .with_columns(
        pl.col('pib_brl').mul(pl.col('pib_2022_pct') + pl.lit(1)),
        pl.col('aaaa_right').cast(str).str.to_date(format='%Y').alias('aaaa')
    )
    .select(
        'municipio_codigo_7',
        'aaaa',
        'pib_brl',
        'uf_codigo',
    )
    .vstack(temp)
    .sort(['aaaa'])
).write_parquet('./municipio_pib.parquet')